In [ ]:
from queens.etl.process import ingest_tables, stage_data, get_data_info
from queens.etl.bootstrap import initialize
from queens import settings as s

initialize(db_path=s.DB_PATH, schema=s.SCHEMA)
ingest_tables("dukes", ["1.1", "6.1"])   # or any small pair you trust
stage_data("dukes")

info_df = get_data_info("dukes", "6.1")
info_df.head()


In [1]:
import subprocess
import sys

# Path to your FastAPI app (adjust as needed)
app_path = "queens.api.app:app"  # module:variable

# Start Uvicorn in the background, listening on all interfaces
process = subprocess.Popen(
    [
        sys.executable, "-m", "uvicorn",
        app_path,
        "--host", "127.0.0.1",
        "--port", "8000",
        "--reload"
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

print(f"Uvicorn started with PID {process.pid}")
print("Visit http://127.0.0.1:8000/docs to explore the API.")


Uvicorn started with PID 17452
Visit http://127.0.0.1:8000/docs to explore the API.


In [ ]:
import queens.settings as s

p = s.LOG_DIR / "queens_logs.log"
p

In [2]:
import requests
import json

# Your API endpoint (adjust port if needed)
base_url = "http://127.0.0.1:8000/data/dukes"

# Query parameters

filters = {
    "year": {"gte": 2014}
}

params = {
    "table_name": "1.1",
    # Example filters: adjust as needed
    "filters": json.dumps(filters) 
}

# Make the GET request
response = requests.get(base_url, params=params)

# Check if request succeeded
#response.raise_for_status()




ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /data/dukes?table_name=1.1&filters=%7B%22year%22%3A+%7B%22gte%22%3A+2014%7D%7D (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002480E098130>: Failed to establish a new connection: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione'))

In [ ]:
from queens import settings as s
#s.set_config(db_path="C:/Users/Alessandro/Desktop/queens/queens.db", 
#             export_path="C:/Users/Alessandro/Desktop/queens/exports/")
print(s.DB_PATH, s.EXPORT_DIR)


In [ ]:
import queens.settings as s
s.LOG_DIR

In [ ]:
from queens.api.app import get_data

dd = get_data(collection="dukes", table_name="2.7", filters=None, limit=1000, cursor=None)

In [ ]:
import json, requests

BASE = "http://127.0.0.1:8000"
COLL = "dukes"
TABLE = "1.1"

# Page 1
curs = None
page = 1
while True:
    r = requests.get(f"{BASE}/data/{COLL}", params={"table_name": TABLE, "limit": 5000, "cursor": curs})
    r.raise_for_status()
    p = r.json()
    print(f"Page {page}: len=", len(p.get("data", [])), "next_cursor=", p.get("next_cursor"))
    curs = p.get("next_cursor")
    page = page  + 1
    if curs is None:
        break
